In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.express as px
from collections import Counter
import time

from bertopic import BERTopic
from itertools import chain
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from umap import UMAP
from hdbscan import HDBSCAN

In [2]:
# email_passage_splits = [len(embedding) for embedding in email_embeddings_np]

# passage_mapping ={}
# j = 0
# for i in range(len(email_passage_splits)):
#     passages_count = email_passage_splits[i]
#     passage_mapping[i] = cluster_labels[j:j+passages_count]
#     j += passages_count

In [5]:
email_df = pd.read_pickle('Email/emails_data_processed.pkl')

In [6]:
flattened_passages = list(chain.from_iterable(email_df['Passages']))

In [7]:
len(flattened_passages)

337215

In [13]:
umap_model = UMAP(n_neighbors=6, min_dist=0.4)
hdbscan_model = HDBSCAN(min_cluster_size=135, gen_min_span_tree=True, prediction_data=True)

In [14]:
# import random
# samples = random.sample(flattened_passages, 50000)

In [15]:
topic_model = BERTopic(n_gram_range=(1, 3), top_n_words=10,
                      umap_model=umap_model, hdbscan_model=hdbscan_model, verbose=True)
topics, probs = topic_model.fit_transform(flattened_passages)

Batches:   0%|          | 0/10538 [00:00<?, ?it/s]

2023-10-14 22:31:59,550 - BERTopic - Transformed documents to Embeddings
2023-10-14 22:38:55,852 - BERTopic - Reduced dimensionality


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

2023-10-14 22:39:21,546 - BERTopic - Clustered reduced embeddings


In [16]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,219155,-1_to_for_the_and,"[to, for, the, and, you, in, of, this, by, our]","[Paid for by Michael Franken for Iowa, IncMich..."
1,0,36745,0_we_our_to_the,"[we, our, to, the, and, in, this, of, you, can]","[And with so much on the ballot this year, fro..."
2,1,4718,1_abortion_roe_wade_roe wade,"[abortion, roe, wade, roe wade, reproductive, ...",[Today marks the 49th anniversary of the Supre...
3,2,4209,2_georgia_stacey_abrams_kemp,"[georgia, stacey, abrams, kemp, warnock, brian...",[–Georgia Democrats --------------------------...
4,3,2821,3_biden_joe_inflation_joe biden,"[biden, joe, inflation, joe biden, the, gas, b...","[Peter, Joe Biden blames everyone but himself ..."
...,...,...,...,...,...
193,192,139,192_andrews for_annie andrews for_andrews_anni...,"[andrews for, annie andrews for, andrews, anni...","[Followinghis clerkship, Mike joined the Salt ..."
194,193,139,193_mullin_kevin mullin_kevin_strahan,"[mullin, kevin mullin, kevin, strahan, kevin m...","[( you, Kevin Mullin==========================..."
195,194,137,194_few things that_few things_maga hat_things...,"[few things that, few things, maga hat, things...","[I have 15 minutes of free time this morning, ..."
196,195,137,195_ken_russell_ken russell_russell is,"[ken, russell, ken russell, russell is, ken ru...","[Thank you, Russell HQ Ken Russell is a Democ..."


In [19]:
topic_model.get_topic_info().to_pickle('Email/email_cluster_bert.pkl')

In [17]:
topic_model.visualize_hierarchy()

In [18]:
topic_model.visualize_hierarchy().write_html('Email/email-bert-clust.html')

In [ ]:
# # Define a list of parameters to try for UMAP
# umap_params = [
#     {'n_neighbors': 3,'min_dist': 0.05},
#     {'n_neighbors': 3,'min_dist': 0.1},
#     {'n_neighbors': 6,'min_dist': 0.05}
# ]

# # Define a list of parameters to try for HDBSCAN
# hdbscan_params = [
#     {'min_cluster_size': 40},
#     {'min_cluster_size': 50}
# ]

# # Loop over the parameter combinations and fit BERTopic models
# for umap_param in umap_params:
#     for hdbscan_param in hdbscan_params:
#         # Create UMAP and HDBSCAN models with the current parameter combination
#         umap_model = UMAP(**umap_param)
#         hdbscan_model = HDBSCAN(**hdbscan_param, gen_min_span_tree=True, prediction_data=True)

#         # Fit a BERTopic model with the current parameter combination
#         model = BERTopic(
#             umap_model=umap_model,
#             hdbscan_model=hdbscan_model,
#             top_n_words=10,
#             language='english',
#             calculate_probabilities=True,
#             verbose=True,
#             n_gram_range=(1, 2)
#         )
#         topics, probs = model.fit_transform(flattened_passages)
#         model.get_topic_info().to_pickle(f'Email/email_model_umap_{umap_param}_hdbscan_{hdbscan_param}.pkl')

#         # Visualize the hierarchy and save the figure to an HTML file
#         fig = model.visualize_hierarchy()
#         fig.write_html(f'Email/email_model_umap_{umap_param}_hdbscan_{hdbscan_param}.html')